<div style="text-align:center"><img src="zillowprojlogo.png"/></div>

<a id='navigation'></a>

<button class="button-save large">[Scenario](#scenario)</button>
<button class="button-save large">[Project Planning](#project-planning)</button>
<button class="button-save large">[Key Findings](#key-findings)</button>
<button class="button-save large">[Tested Hypotheses](#tested-hypotheses)</button>
<button class="button-save large">[Take Aways](#take-aways)</button>
<button class="button-save large">[Data Dictionary](#data-dictionary)</button>
<button class="button-save large">[Workflow](#workflow)</button>

<div class="alert alert-block alert-info"><a name="scenario"></a><h1><i class="fas fa-home"></i> Scenario</h1></div>
Selling homes in our new normal has just gotten easier with Zillow Offers®. Now home owners can hand over the burden of selling their property, by selling directly to us based on our state of the art Zestimate score.

The accuracy and integrity of our Zestimate score is of high importance. As a junior data scientists on the Zillow data science team, we are tasked with uncovering what drivers most affect the validity of the Zestimate score. This is measured by our target variable: `logerror`, which is the difference between Zillow's estimated Zestimate and actual sale price. 
>`logerror` = log (Zestimate) − log (ActualSalePrice)

### Project Goal: 
The goal for this project is to create a model that will accurately predict the Zestimate’s `logerror`. By doing so, we will uncover what features available on the Zillow Dataset are driving the amount of error.


In [5]:
# Imports needed

import pandas as pd
import numpy as np
import os
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from env import host, user, password
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.cluster import KMeans

import seaborn as sns
import matplotlib.pyplot as plt
import wrangle

# modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor

## Acquire the Data

In [9]:
# Acquire the data from SQL

df = wrangle1.zillow17()
df

NameError: name 'wrangle1' is not defined

### Acquire Takeaways:
- Massive dataset
- Many columns are primarily nulls/NaNs
- Many columns are redundant

## Prep the Data

In [3]:
# Dropping unnecessary/redundant columns, limiting

df = wrangleag.wrangle_zillow()

NameError: name 'wrangleag' is not defined

In [4]:
df.info()

NameError: name 'df' is not defined

In [ ]:
df.describe().T

### Data Prep Takeaways:
- Dropped unnecessary/redundant columns: 'id', 'calculatedbathnbr', 'finishedsquarefeet12', 'fullbathcnt', 'heatingorsystemtypeid',                 'propertycountylandusecode', 'propertylandusetypeid','propertyzoningdesc', 'censustractandblock', 'propertylandusedesc', 
  'heatingorsystemdesc', 'assessmentyear', 'regionidcounty'  
- Set our outliers to taxvaluedollarcnt < 5_000_000 and calculatedfinishedsquarefeet < 12500
- Dropped columns that had >60% nulls, dropped rows that had >70% nulls
- Dropped a total of 45 columns and 5756 rows
- Still a large dataset

In [ ]:
## Explore the data